# Log Analysis and Visulization

In [ ]:
import utils
import os
from os import path
import pickle
import pandas as pd

In [ ]:
if __name__ == "__main__":
    
    with open('./tmp/shard.pkl', 'rb') as f:
        shard = pickle.load(f)
    with open('./tmp/param.pkl', 'rb') as f:
        param = pickle.load(f)
        
    date = shard[0].iloc[0]['timestamp'].strftime("%Y-%m-%d %H:%M:%S").split()[0].replace("-","_")     
    fig_dir = "../../graphs/test_logs/ostn/" + date + "/part/"
    html_dir = "../../docs/graphs/test_logs/ostn/" + date + "/part/"
    
    data = [html_dir, fig_dir]
    for directory in data:
        if not path.exists(directory):
            try:
                os.makedirs(directory)
            except:
                print("Could not make directory")
                exit(1)
    
    utils.set_config(param)
    config = utils.config
    ## get the index of the new chain started
    index = []
    for s in shard:
        i = s[s['time_per_block'] < 0].index.tolist()   
        if len(i) == 0:
            break
        i.insert(0,0)
        i.append(len(s))
        index.append(i)
    
    if len(index) == 0:
        new = pd.concat(shard)
        new.reset_index(inplace = True, drop = True)
        utils.visualization(new, fig_dir, html_dir, config)
    else:    
        for i in range(len(index[0])-1):
            df = []
            for s in range(len(shard)):
                df.append(shard[s].iloc[index[s][i]+1:index[s][i+1]])
            new = pd.concat(df)

            fig_path = fig_dir + "chain_" + str(i) + "/"
            if not path.exists(fig_path):
                os.makedirs(fig_path)

            utils.visualization(new, fig_path, html_dir, config)